In [2]:
import pandas as pd
import requests
client_secret = "0ffcb78d32ea42d5bb37456f919f4809"
client_id = "0b3bd3d84e824a68a53325cb16e8eb57"

def get_acces_token(client_id, client_secret):
    """retourne un json (dict)"""
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": f"{client_id}",
        f"client_secret": f"{client_secret}"
    }

    response = requests.post(url, headers=headers, data=data)

    return response.json()

accessToken = get_acces_token(client_id, client_secret)["access_token"]

def get_playlist(accessToken, playlist_id):
    playlist = requests.get(
        url="https://api.spotify.com/v1/playlists/" + playlist_id,
        headers={
            "Authorization": 'Bearer ' + accessToken
        }
    )
    return playlist.json() 

def get_several_tracks_features(accessToken, songs_id : list) -> list:
    """maximum 100 ids"""
    if (len(songs_id) > 100):
        raise ValueError("Too Many Songs")
    songs_id_string = ",".join(songs_id)
    songs_features = requests.get(
        url="https://api.spotify.com/v1/audio-features?ids=" + songs_id_string,
        headers={
            "Authorization": 'Bearer ' + accessToken
        }
    )
    return songs_features.json()

def get_playlist_tracks_features(accessToken, playlist, country): 
    """params: spotify playlist json object, returns a pandas dataframe"""
    songs_id = []
    songs_titles = []
    songs_artists = []
    for track in playlist["tracks"]["items"]:
        info = track["track"]
        songs_titles.append(info["name"])
        songs_artists.append([artist["id"] for artist in info["artists"]])
        songs_id.append(info["id"])
    songs_features_raw = get_several_tracks_features(accessToken, songs_id)
    songs_features = songs_features_raw["audio_features"]
    #danceabilities = [song["danceability"] for song in songs_features]

    if len(songs_features) != len(songs_id):
        print("Not all songs have been succesfully analysed, aborted")

    else :
        temp = pd.DataFrame({"title" : songs_titles, "artists_id": songs_artists})
        songs_df = pd.DataFrame(songs_features)
        songs_df = pd.concat([temp, songs_df], axis = 1)
        songs_df["country"] = country
        return songs_df

def get_top50_features(playlists_df, accessToken):
    features_df = pd.DataFrame()
    for ind in playlists_df.index:
        playlist_id = playlists_df["playlist_id"][ind]
        country = playlists_df["target_country"][ind]
        playlist = get_playlist(accessToken, playlist_id)
        new_playlist_df = get_playlist_tracks_features(accessToken, playlist, country)
        features_df = pd.concat([features_df, new_playlist_df], axis=0)
    return features_df


Date d'extraction : 11/11/2024

In [9]:
playlist_df = pd.read_csv("./data/cleaned_top50_playlists_data.csv")
features_df = get_top50_features(playlist_df, accessToken)

features_df

,title,artists_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,country
0,IA,[78F6Xkd46aMHkr4AP60TXY],0.706,0.631,0,-5.812,0,0.0366,0.6560,0.000000,...,0.289,111.930,audio_features,7lcbpfihslisNObKdHjyLo,spotify:track:7lcbpfihslisNObKdHjyLo,https://api.spotify.com/v1/tracks/7lcbpfihslis...,https://api.spotify.com/v1/audio-analysis/7lcb...,171429,4,Andorra
1,Q U E V A S H A C E R H O Y ?,"[3E12tRURRvPfHz0hAMCFYc, 54seKvtsZauR1iauN0ptpo]",0.560,0.681,6,-6.304,0,0.2440,0.1030,0.000000,...,0.388,155.079,audio_features,0Sr7ssScx54yxdM2ols7nW,spotify:track:0Sr7ssScx54yxdM2ols7nW,https://api.spotify.com/v1/tracks/0Sr7ssScx54y...,https://api.spotify.com/v1/audio-analysis/0Sr7...,223320,4,Andorra
2,DIABLA,[30Ml7nTMkXYyb0UoLwtmCc],0.543,0.587,10,-7.709,0,0.4970,0.7860,0.000000,...,0.723,174.075,audio_features,00EWeXw1RLUwCMw8pxtXAE,spotify:track:00EWeXw1RLUwCMw8pxtXAE,https://api.spotify.com/v1/tracks/00EWeXw1RLUw...,https://api.spotify.com/v1/audio-analysis/00EW...,217946,4,Andorra
3,DEGENERE 👑 (feat. benny blanco),"[7iK8PXO48WeuP03g8YR51W, 5CiGnKThu5ctn9pBxv7DGa]",0.744,0.709,6,-5.778,0,0.2840,0.0501,0.000000,...,0.712,194.055,audio_features,0qeIFEGRQqHoaJtLCl0U73,spotify:track:0qeIFEGRQqHoaJtLCl0U73,https://api.spotify.com/v1/tracks/0qeIFEGRQqHo...,https://api.spotify.com/v1/audio-analysis/0qeI...,130880,4,Andorra
4,Me has invitado a bailar,[0CVOcYvRcSvOXyuR4YGKaC],0.665,0.544,10,-3.648,1,0.1150,0.0129,0.000000,...,0.625,163.948,audio_features,3tMftGMXTPbQlmjh4FzXpZ,spotify:track:3tMftGMXTPbQlmjh4FzXpZ,https://api.spotify.com/v1/tracks/3tMftGMXTPbQ...,https://api.spotify.com/v1/audio-analysis/3tMf...,192906,4,Andorra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,shhhhhhh..,"[30eFAXoU2kTjJPf2cq80B8, 3diftVOq7aEIebXKkC34oR]",0.739,0.537,2,-7.241,0,0.0394,0.2510,0.000000,...,0.379,137.970,audio_features,4jWmhzf53Q0ZvZMuGdiR9x,spotify:track:4jWmhzf53Q0ZvZMuGdiR9x,https://api.spotify.com/v1/tracks/4jWmhzf53Q0Z...,https://api.spotify.com/v1/audio-analysis/4jWm...,230676,4,Vietnam
46,Simp Gái 808,[6TITnFVRcl0AcZ4syE7Toe],0.778,0.514,8,-12.450,0,0.1970,0.3550,0.000535,...,0.224,94.994,audio_features,5J3m0taZ0W1E9HTE0MwcSv,spotify:track:5J3m0taZ0W1E9HTE0MwcSv,https://api.spotify.com/v1/tracks/5J3m0taZ0W1E...,https://api.spotify.com/v1/audio-analysis/5J3m...,161948,4,Vietnam
47,Say Yes (Vietnamese Version),"[61ydMyohF67xBZmWSPA3YQ, 5mPbBOU2KMFfStepNZ027...",0.679,0.747,4,-6.917,1,0.0575,0.2860,0.000000,...,0.505,100.073,audio_features,7mbWwlHgukuuW8r7Nc3FHv,spotify:track:7mbWwlHgukuuW8r7Nc3FHv,https://api.spotify.com/v1/tracks/7mbWwlHgukuu...,https://api.spotify.com/v1/audio-analysis/7mbW...,223200,4,Vietnam
48,"REGRET (feat. Quân A.P, Pháp Kiều, Quang Trung...","[4wZ6awunqaaVLl5j0WpDFu, 2NRcG7E1j2sSi8vnUzCcp...",0.548,0.636,8,-7.157,1,0.0551,0.1310,0.000000,...,0.114,155.889,audio_features,3JIfmjJbanxJkwaPzYa2gN,spotify:track:3JIfmjJbanxJkwaPzYa2gN,https://api.spotify.com/v1/tracks/3JIfmjJbanxJ...,https://api.spotify.com/v1/audio-analysis/3JIf...,252308,4,Vietnam
